In [1]:
# Automatically reload all imported modules
%load_ext autoreload
%autoreload 2
# Allow to pan and zoom graphs & charts
%matplotlib widget

# Categorical Attributes

## Categorical Attributes

**Let's switch to a different dataset (a toy one)**

<center><img src="assets/weather.jpg" style="height: 400px;"></center>

We want to train a model to choose whether to go out and play, based on weather conditions


## Loading the Data

**The dataset is in the `weather.csv` files from the `data` folder**

In [2]:
!ls data

lr_test.txt  lr_train.txt  real_estate.csv  weather.csv


In [3]:
import pandas as pd
data = pd.read_csv('data/weather.csv', sep=',')
data.head()

,outlook,temperature,humidity,windy,play
0,sunny,85,85,False,no
1,sunny,80,90,True,no
2,overcast,83,86,False,yes
3,rainy,70,96,False,yes
4,rainy,68,80,False,yes


* Several attributes _do not have a numeric value_
* Instead, their value is discrete with no clear ordering, i.e. _categorical_

**We need a _numeric encoding_ to handle this data with linear models**

## Encoding Binary Attributes

**Binary attributes can be encoded with the values 0 and 1**

This is the case for the columns "windy" and "play"

* First, we tell pandas that the columns have a categorical type

In [4]:
windy = data['windy'].astype('category')
play = data['play'].astype('category')
windy.head()

0    False
1     True
2    False
3    False
4    False
Name: windy, dtype: category
Categories (2, object): [False, True]

* Categorical data is still _displayed as a string_
* ...But internally it is _encoded as an integer_

## Encoding Binary Attributes

**Next, we replace the values with their integer code**

We will store the results in a copy of the original table

In [5]:
data2 = data.copy() # We prepare a cop for the numeric encodings
data2['windy'] = windy.cat.codes
data2['play'] = play.cat.codes
data2.head()

,outlook,temperature,humidity,windy,play
0,sunny,85,85,0,0
1,sunny,80,90,1,0
2,overcast,83,86,0,1
3,rainy,70,96,0,1
4,rainy,68,80,0,1


* Now it is apparent that "windy" and "play" have become numbers

## Encoding Discrete Attributes

**We could use the same approach for discrete attribute in general**

E.g. for the attribute "outlook" in our table

* That would yield a numeric _integer_ encoding
* ...Which implies an ordering among the values (e.g. rainy < overcast < sunny)
* When no such ranking exists, this is a bad idea

**In these cases, it is better to adopt a one-hot encoding**

* We introduce a column for each value $v_k$ of the attribute $xj$
* The column contains a 1 iff $x_j = v_k$, and 0 otherwise
* We remove the original attribute

For example, "sunny | sunny | overcast" becomes:

rainy | overcast | sunny
:----:|:--------:|:-----:
  0   |    0     |   1
  0   |    0     |   1
  0   |    1     |   0

## Encoding Discrete Attributes

**We can obtain a one-hot encoding in pandas via the `get_dummies` method**

In [6]:
data2 = pd.get_dummies(data2)
data2.head()

,temperature,humidity,windy,play,outlook_overcast,outlook_rainy,outlook_sunny
0,85,85,0,0,0,0,1
1,80,90,1,0,0,0,1
2,83,86,0,1,1,0,0
3,70,96,0,1,0,1,0
4,68,80,0,1,0,1,0


* The method by default processes all columns with categorical or object type
* strings in csv files are often parsed as "object" columns
* In truth, `get_dummies` can also handle the special case of binary variables
* ...But I wanted to show you how to obtain an integer encoding, too :-)

# Logistic Regression

## Logistic Regression

**Our goal is to predict the value of "play", i.e. _a categorical attribute_**

Hence, we are dealing with _a classification problem_ and we need to make a few changes 

* We compute the output of the linear model as usual:
$$
g(x; w) = \sum_{j=1} w_j x_j + w_0
$$ 
* Then we feed it to a _logistic function_:
$$
f(x; w) = \frac{1}{1 + e^{-g(x; w)}}
$$

**Note that here:**

* The basis functions are $\phi_j(x) = x_j$
* There is an extra basis function $\phi_0(x) = 1$ for the offset
* This transformation can always be performed via a precomputation step

## Logistic Regression

**The logistic function is a type of _sigmoid_ function**

In [7]:
import numpy as np
from matplotlib import pyplot as plt
x = np.linspace(-10, 10, 100)

plt.figure(figsize=(9, 3))
plt.plot(x, 1 / (1 + np.exp(-x)))
plt.grid(':')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

* Due to its use, this approach is known as _logistic regression_

## Logistic Regression

**Why using the logistic function?**

* We can view the model output as a probability distribution
* Specifically, as the probability of the class being "1"

With this convention, the target can also be interpreted as a probability

In [8]:
data2['play'].head()

0    0
1    0
2    1
3    1
4    1
Name: play, dtype: int8

We view:

* $y_i = 0$ as "the probability of the class being 1 is equal to 0"
* $y_i = 1$ as "the probability of the class being 1 is equal to 1"

## Likelihood Function

**We now assume that our model $f(x; w)$ is _the source of our data_**

* When we see $y_i = 1$, we view it as "the model has generated a 1"
* When we see $y_i = 0$, we view it as "the model has generated a 0"


**With this, we can estimate _the probability of generating our data_**

This is given by the probability of:

* ...generating a 1 for all examples where the class is 1:
$$
\prod_{y_i=1} f(x_i; w)
$$
* ...times that of generating a 0 for all examples where the class is 0:
$$
\prod_{y_i=0} (1-f(x_i; w))
$$

## Likelihood Function

**Overall, for a given dataset we get:**

$$
L(w) = \prod_{y_i=1} f(x_i; w) \prod_{y_i=0} (1-f(x_i; w))
$$

* The function is akin to a probability, but is _associated to our model_, not to the data itself
* ...And in fact _depends on the parameters $w$_

To make the distinction clearer, we call it a _likelihood function_

**The likelihood is typically expressed in log form**

$$
\log L(w) = \sum_{i=1}^m y_i \log f(x_i; w) + (1-y_i) \log (1 - f(x; w))
$$

This expression happens to be the negative of the [_binary cross-entropy_](https://en.wikipedia.org/wiki/Cross_entropy)

## Maximum Likelihood Estimation

**Why is this relevant?**

We want to train a model that is a _likely source for our data_

* This means that we can choose the weights by solving:
$$
\text{argmax}_w \log L(w)
$$
* I.e. to _maximize the likelihood_ of the data

With change of sign we get the equivalent formulation:
$$
\text{argmin}_w - \log L(w)
$$
* I.e. we want to _minimize the binary cross-entropy_
* ...Meaning that the cross-entropy is going to be our _loss function_


## Maximum Likelihood Estimation

**MLE is very important in many Machine Learning approaches**

* It provides an mathematical foundation and intuition for the training process
  - I.e. we want to obtain a model the could plausibly be a source of the data
* It applies to linear regression, too!
  - The MSE is mapped 1-1 to a likelihood function...
  - ...Assuming additive, Gaussian, noise with uniform variance

**How do we solve the training problem?**

$$
\text{argmin}_w - \log L(w)
$$

* We can no longer reduce it to a system of linear equations
* ...But it still has a unique minimizer!

So it can be solved (e.g.) via gradient descent


## Using Logistic Regression

**Using Logistic Regression in scikit-learn is actually easy**

We begin by splitting input/output data as usual:

In [9]:
cols_in = [c for c in data2.columns if c != 'play']

X = data2[cols_in]
y = data2['play']
y.head() # We have a table here, but a vector would also work

0    0
1    0
2    1
3    1
4    1
Name: play, dtype: int8

Then the training and test set: 

In [10]:
from sklearn.model_selection import train_test_split
X_tr, X_ts, y_tr, y_ts = train_test_split(X, y, test_size=0.34, random_state=0)

## Using Logistic Regression

**Then, we build a `LogisticRegression` model**

In [11]:
from sklearn.linear_model import LogisticRegression

m = LogisticRegression()

...And we call the `fit` method as usual:

In [12]:
m.fit(X_tr, y_tr);

Finally, we can obtain out predictions:

In [13]:
y_pred_tr = m.predict(X_tr)
y_pred_ts = m.predict(X_ts)

## A Better Look at the Predictions

**By default, the prediction is the class with the largest probability**

In [14]:
y_pred_tr

array([0, 1, 0, 0, 1, 0, 0, 1, 1], dtype=int8)

* If we are interested in the raw probability values...
* ...We can call the `predict_proba` method:

In [15]:
y_prob_tr = m.predict_proba(X_tr)
y_prob_tr[:5]

array([[0.726788  , 0.273212  ],
       [0.3675285 , 0.6324715 ],
       [0.77878844, 0.22121156],
       [0.77593317, 0.22406683],
       [0.48442042, 0.51557958]])

* Scikit-learn gives us the predicted probability of both classes
* Hence, we get two separate columns

## Evaluation

**We can evaluate the results using metrics**

There are four basic metrics for binary classification:

* Number of _True Positives_, i.e. $\mathit{TP} = \sum_{y_i=1} \tilde{f}(x_i; w)$
* Number of _True Negatives_, i.e. $\mathit{TN} = \sum_{y_i=0} (1 - \tilde{f}(x_i; w))$
* Number of _False Positives_, i.e. $\mathit{FP} = \sum_{y_i=0} \tilde{f}(x_i; w)$
* Number of _False Negatives_, i.e. $\mathit{FN} = \sum_{y_i=1} (1 - \tilde{f}(x_i; w))$

In all cases $\tilde{f}(x_i; w)$ is the most probable class for the example $x_i$



## Evaluation

**From these we can derive a few more complex metrics**

The model (binary) _accuracy_ is defined as:
$$
\mathit{ACC} = \frac{TP + TN}{m}
$$
* I.e. the fraction of examples that is _correctly classified_
* The accuracy ranges over the interval $[0, 1]$

In [16]:
from sklearn.metrics import accuracy_score

print(f'Accuracy on the training set: {accuracy_score(y_tr, y_pred_tr):.3}')
print(f'Accuracy on the test set: {accuracy_score(y_ts, y_pred_ts):.3}')

Accuracy on the training set: 0.778
Accuracy on the test set: 0.8


## Evaluation

**From these we can derive a few more complex metrics**

The $F_1$ score is given by:
$$
\mathit{F1} = \frac{TP}{TP + \frac{1}{2}(FP+FN)}
$$
* The F1 score ranges in the interval $[0, 1]$
* F1 = 1 iff all positives are found and no negative is misclassified
* F1 = 0 iff no positive is correctly classified

In [17]:
from sklearn.metrics import f1_score

print(f'F1 on the training set: {f1_score(y_tr, y_pred_tr):.3}')
print(f'F1 on the training set: {f1_score(y_ts, y_pred_ts):.3}')

F1 on the training set: 0.75
F1 on the training set: 0.889


## Evaluation

**...Or we can plot all basic metrics via a _confusion matrix_**

Here's the one for the training set:

In [19]:
from sklearn.metrics import plot_confusion_matrix
plot_confusion_matrix(m, X_tr, y_tr, display_labels=play.cat.categories, cmap='Blues');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Evaluation

**...Or we can plot all basic metrics via a _confusion matrix_**

...And here for the test set:

In [20]:
from sklearn.metrics import plot_confusion_matrix
plot_confusion_matrix(m, X_ts, y_ts, display_labels=play.cat.categories, cmap='Blues');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …